# Simple Extract Transform Load (ETL)

In [232]:
import glob                        
import pandas as pd                
import xml.etree.ElementTree as ET  
from datetime import datetime

In [233]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip


n
--2023-05-18 14:49:21--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
n
n
n
n
n
n
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2023-05-18 14:49:21 (511 MB/s) - ‘datasource.zip’ saved [4249/4249]



In [234]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
replace dealership_data/used_car_prices1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices3.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices1.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices1.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices2.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices3.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [235]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

# Extract

json extract function

In [236]:
def extract_json(file_to_extract):
  df = pd.read_json(file_to_extract, lines = True)
  return df

csv extract function

In [237]:
def extract_csv(file_to_extract):
  df = pd.read_csv(file_to_extract)
  return df

XML extract function

In [238]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

Extraction function

In [239]:
def extract():
   extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
   #csv files
   for csv in glob.glob("dealership_data/*.csv"):
    extracted_data = extracted_data.append(extract_csv(csv), ignore_index = True)
  #json files
   for js in glob.glob("dealership_data/*.json"):
    extracted_data = extracted_data.append(extract_json(js), ignore_index = True)
  #xml files
   for xml in glob.glob("dealership_data/*.xml"):
    extracted_data = extracted_data.append(extract_from_xml(xml),ignore_index= True)
   return extracted_data

# Transform function

In [240]:

def transform(data):
        data['price'] = round(data.price, 2)
        return data


Load function

In [241]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

Logging function

In [242]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 

ETL Process

In [243]:

log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data.head()
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")

<ipython-input-239-df4ed772189c>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_csv(csv), ignore_index = True)
<ipython-input-239-df4ed772189c>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_csv(csv), ignore_index = True)
<ipython-input-239-df4ed772189c>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_csv(csv), ignore_index = True)
<ipython-input-239-df4ed772189c>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_json(js), ignore_index = True)
<ipy

In [244]:
extracted_data.head()

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
